In [10]:

import pandas as pd

# Mount Google Drive





In [30]:
# Path to the file on Google Drive
file_path = 'Philips_Employee_Reviews_from_AmbitionBox.csv'

# Load the dataset
data = pd.read_csv(file_path)

# Now you can continue with the rest of your code using the 'data' DataFrame

In [31]:
import pandas as pd


# Preprocess the text data
# Remove punctuation, convert text to lowercase, and remove stopwords
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Check if text is a string
    if isinstance(text, str):
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
        # Convert text to lowercase
        text = text.lower()
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

    return text

data['Likes'] = data['Likes'].apply(preprocess_text)
data['Dislikes'] = data['Dislikes'].apply(preprocess_text)

# Explore the dataset
print(data.head())
print(data.info())


                                               Title                Place  \
0                           Software Developer Cloud              Chennai   
1                                       Team Manager              Chennai   
2  Engineering Manager Engineering Head Mechanica...    Pune, Maharashtra   
3                                  Marketing Manager  Mumbai, Maharashtra   
4                                Product Costing BPE              Chennai   

    Job_type                       Department        Date  Overall_rating  \
0  Full Time  Software Development Department  4 Sep 2023             1.0   
1  Full Time    Retail & B2C Sales Department  2 Sep 2023             3.0   
2  Full Time        Connected care Department  0 Sep 2023             3.0   
3  Full Time             Marketing Department  7 Sep 2023             1.0   
4  Full Time            Operations Department  3 Sep 2023             5.0   

   work_life_balance  skill_development  salary_and_benefits  job_security

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Generate word clouds for Likes and Dislikes
likes_text = ' '.join(data['Likes'].apply(lambda x: str(x)))
dislikes_text = ' '.join(data['Dislikes'].apply(lambda x: str(x)))


likes_wordcloud = WordCloud(width=800, height=400, background_color='white').generate(likes_text)
dislikes_wordcloud = WordCloud(width=800, height=400, background_color='white').generate(dislikes_text)

# Visualize word clouds
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(likes_wordcloud, interpolation='bilinear')
plt.title('Likes Word Cloud')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(dislikes_wordcloud, interpolation='bilinear')
plt.title('Dislikes Word Cloud')
plt.axis('off')

plt.show()


ModuleNotFoundError: No module named 'wordcloud'

In [35]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

# Load the dataset
file_path = 'Philips_Employee_Reviews_from_AmbitionBox.csv'
data = pd.read_csv(file_path)

# Preprocess the text data
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Check if text is a string
    if isinstance(text, str):
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
        # Convert text to lowercase
        text = text.lower()
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

data['Likes'] = data['Likes'].apply(preprocess_text)
data['Dislikes'] = data['Dislikes'].apply(preprocess_text)

# Combine Likes and Dislikes into a single text column
data['Text'] = data['Likes'] + ' ' + data['Dislikes']

# Drop rows with missing values in both 'Text' and 'Overall_rating' columns
data.dropna(subset=['Text', 'Overall_rating'], inplace=True)

# Define thresholds for sentiment categories
positive_threshold = 5
negative_threshold = 3

# Map ratings to sentiment categories
data['Sentiment'] = data['Overall_rating'].apply(lambda x: 'positive' if x >= positive_threshold else ('negative' if x <= negative_threshold else 'neutral'))

# Define features and target
X = data['Text']
y = data['Sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using Bag-of-Words
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vect, y_train)

# Predict on the test set
y_pred = model.predict(X_test_vect)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Define the directory path for the model folder
model_dir = 'model'

# Create the model directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Define the file paths for saving the trained model and CountVectorizer instance
model_file_path = os.path.join(model_dir, 'sentiment_analysis_model_v1.pkl')
vectorizer_file_path = os.path.join(model_dir, 'count_vectorizer.pkl')

# Save the trained model and CountVectorizer instance to the file paths
joblib.dump(model, model_file_path)
joblib.dump(vectorizer, vectorizer_file_path)

print("Trained model and CountVectorizer instance saved successfully to:", model_file_path, "and", vectorizer_file_path)


Accuracy: 0.4976958525345622
Classification Report:
               precision    recall  f1-score   support

    negative       0.47      0.31      0.38        54
     neutral       0.43      0.41      0.42        64
    positive       0.54      0.66      0.59        99

    accuracy                           0.50       217
   macro avg       0.48      0.46      0.46       217
weighted avg       0.49      0.50      0.49       217

Trained model and CountVectorizer instance saved successfully to: model\sentiment_analysis_model_v1.pkl and model\count_vectorizer.pkl


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report

# # Combine Likes and Dislikes into a single text column
# data['Text'] = data['Likes'] + ' ' + data['Dislikes']

# # Drop rows with missing values in both 'Text' and 'Overall_rating' columns
# data.dropna(subset=['Text', 'Overall_rating'], inplace=True)

# # Define thresholds for sentiment categories
# positive_threshold = 6
# negative_threshold = 2

# # Map ratings to sentiment categories
# data['Sentiment'] = data['Overall_rating'].apply(lambda x: 'positive' if x >= positive_threshold else ('negative' if x <= negative_threshold else 'neutral'))

# # Define features and target
# X = data['Text']
# y = data['Sentiment']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Vectorize the text data using Bag-of-Words
# vectorizer = CountVectorizer()
# X_train_vect = vectorizer.fit_transform(X_train)
# X_test_vect = vectorizer.transform(X_test)

# # Initialize and train the Logistic Regression model
# model = LogisticRegression(max_iter=1000)
# model.fit(X_train_vect, y_train)

# # Predict on the test set
# y_pred = model.predict(X_test_vect)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)
# print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7419354838709677
Classification Report:
               precision    recall  f1-score   support

    negative       0.29      0.18      0.22        22
     neutral       0.43      0.09      0.15        32
    positive       0.79      0.94      0.86       163

    accuracy                           0.74       217
   macro avg       0.50      0.41      0.41       217
weighted avg       0.68      0.74      0.69       217



In [27]:
# Assuming you have already trained and saved the model

# New text input
new_text = "wow i like it"

# Combine likes and dislikes if needed
# new_text_combined = preprocess_text(new_text)

# Vectorize the new text input
new_text_vect = vectorizer.transform([new_text])

# Predict the sentiment category
predicted_sentiment = model.predict(new_text_vect)[0]

print("Predicted Sentiment:", predicted_sentiment)


Predicted Sentiment: positive


In [28]:
# import joblib
# import os

# # Define the directory path for the model folder
# model_dir = 'model'

# # Create the model directory if it doesn't exist
# if not os.path.exists(model_dir):
#     os.makedirs(model_dir)

# # Define the file path for saving the trained model
# model_file_path = os.path.join(model_dir, 'sentiment_analysis_model_v1.pkl')

# # Save the trained model to the file path
# joblib.dump(model, model_file_path)

# print("Trained model saved successfully to:", model_file_path)



Trained model saved successfully to: model\sentiment_analysis_model_v1.pkl


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define aspects
aspects = ['work_life_balance', 'salary_and_benefits']

# Preprocess the text data
# Assuming the data is already preprocessed and contains 'Likes' and 'Dislikes' columns
data.dropna(subset=['Likes', 'Dislikes', 'work_life_balance', 'salary_and_benefits'], inplace=True)

# Combine Likes and Dislikes into a single text column for each aspect
data['work_life_balance_text'] = data['Likes'] + ' ' + data['Dislikes']
data['salary_and_benefits_text'] = data['Likes'] + ' ' + data['Dislikes']

# Define features and target for each aspect
X_work_life_balance = data['work_life_balance_text']
X_salary_and_benefits = data['salary_and_benefits_text']
y_work_life_balance = data['work_life_balance']
y_salary_and_benefits = data['salary_and_benefits']

# Split the data into training and testing sets for each aspect
X_train_wlb, X_test_wlb, y_train_wlb, y_test_wlb = train_test_split(X_work_life_balance, y_work_life_balance, test_size=0.2, random_state=42)
X_train_sal, X_test_sal, y_train_sal, y_test_sal = train_test_split(X_salary_and_benefits, y_salary_and_benefits, test_size=0.2, random_state=42)


In [19]:
# Vectorize the text data using TF-IDF for each aspect
vectorizer_wlb = TfidfVectorizer()
X_train_vect_wlb = vectorizer_wlb.fit_transform(X_train_wlb)
X_test_vect_wlb = vectorizer_wlb.transform(X_test_wlb)

vectorizer_sal = TfidfVectorizer()
X_train_vect_sal = vectorizer_sal.fit_transform(X_train_sal)
X_test_vect_sal = vectorizer_sal.transform(X_test_sal)

# Initialize and train Logistic Regression models for each aspect
model_wlb = LogisticRegression(max_iter=1000)
model_wlb.fit(X_train_vect_wlb, y_train_wlb)

model_sal = LogisticRegression(max_iter=1000)
model_sal.fit(X_train_vect_sal, y_train_sal)


LogisticRegression(max_iter=1000)

In [20]:
# Predict on the test set for each aspect
y_pred_wlb = model_wlb.predict(X_test_vect_wlb)
y_pred_sal = model_sal.predict(X_test_vect_sal)

# Evaluate the performance of each model
accuracy_wlb = accuracy_score(y_test_wlb, y_pred_wlb)
accuracy_sal = accuracy_score(y_test_sal, y_pred_sal)

print("Work-life balance aspect:")
print("Accuracy:", accuracy_wlb)
print("Classification Report:\n", classification_report(y_test_wlb, y_pred_wlb))

print("\nSalary and benefits aspect:")
print("Accuracy:", accuracy_sal)
print("Classification Report:\n", classification_report(y_test_sal, y_pred_sal))


Work-life balance aspect:
Accuracy: 0.4009216589861751
Classification Report:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        20
         2.0       0.00      0.00      0.00        13
         3.0       0.00      0.00      0.00        26
         4.0       0.26      0.16      0.20        61
         5.0       0.45      0.79      0.57        97

    accuracy                           0.40       217
   macro avg       0.14      0.19      0.15       217
weighted avg       0.27      0.40      0.31       217


Salary and benefits aspect:
Accuracy: 0.3087557603686636
Classification Report:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        17
         2.0       0.00      0.00      0.00        13
         3.0       0.14      0.11      0.12        35
         4.0       0.34      0.47      0.40        75
         5.0       0.33      0.36      0.34        77

    accuracy            

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

In [21]:
# Example text for testing work-life balance aspect
text_wlb = "The company offers flexible working hours and encourages employees to maintain a healthy work-life balance."

# Example text for testing salary & benefits aspect
text_sal = "The salary is competitive, and the benefits package includes healthcare, retirement plans, and bonuses."

# Vectorize the text using TF-IDF
text_vect_wlb = vectorizer_wlb.transform([text_wlb])
text_vect_sal = vectorizer_sal.transform([text_sal])

# Predict sentiment for each aspect
sentiment_wlb = model_wlb.predict(text_vect_wlb)
sentiment_sal = model_sal.predict(text_vect_sal)
# Set threshold for positive and negative sentiments
threshold_positive = 4  # Ratings above this threshold are considered positive
threshold_negative = 2  # Ratings below this threshold are considered negative

# Function to classify sentiment based on threshold
def classify_sentiment(rating, threshold_pos, threshold_neg):
    if rating > threshold_pos:
        return "Positive"
    elif rating < threshold_neg:
        return "Negative"
    else:
        return "Neutral"

# Predict sentiment for each aspect and classify based on threshold
sentiment_wlb = classify_sentiment(sentiment_wlb, threshold_positive, threshold_negative)
sentiment_sal = classify_sentiment(sentiment_sal, threshold_positive, threshold_negative)

print("Sentiment for Work-Life Balance Aspect:", sentiment_wlb)
print("Sentiment for Salary & Benefits Aspect:", sentiment_sal)



Sentiment for Work-Life Balance Aspect: Positive
Sentiment for Salary & Benefits Aspect: Positive


In [22]:
import joblib
import os

# Define the directory path for the model folder
model_dir = 'model'

# Create the model directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Define the file path for saving the trained models
model_wlb_filename = os.path.join(model_dir, 'sentiment_analysis_model_wlb.pkl')
model_sal_filename = os.path.join(model_dir, 'sentiment_analysis_model_sal.pkl')

# Save the trained models to files
joblib.dump(model_wlb, model_wlb_filename)
joblib.dump(model_sal, model_sal_filename)

print("Models saved successfully!")



Models saved successfully!
